# Nemotron Next 8B LoRA Fine-tuning with FiQA Dataset

This notebook demonstrates LoRA fine-tuning of [NVIDIA Nemotron Next 8B](https://huggingface.co/nvidia/Nemotron-Next-8B) on the [FiQA dataset](https://huggingface.co/datasets/explodinggradients/fiqa) for financial question answering.

## Table of Contents

1. [Setup & Environment](#setup)
2. [Model Loading](#model-loading)
3. [Dataset Loading & Preprocessing](#dataset)
4. [Baseline Evaluation](#baseline)
5. [LoRA Configuration](#lora-config)
6. [LoRA Training](#training)
7. [Fine-tuned Evaluation](#evaluation)
8. [Visualization & Analysis](#visualization)

---

## GPU Requirements

⚠️ **This notebook requires a GPU with 24GB+ VRAM** (A100, H100, or RTX 4090 recommended)

| Phase | GPU Required | Time Estimate |
|-------|--------------|---------------|
| Model Loading | ✅ Yes | 2-5 min |
| Dataset Prep | ✅ Yes | 10-25 min |
| Baseline Eval | ✅ Yes | 30-60 min |
| LoRA Training | ✅ Yes | 3-6 hours |
| Final Eval | ✅ Yes | 30-60 min |


<a name="setup"></a>
## 1. Setup & Environment

First, let's verify our environment and import required libraries.


In [ ]:
# GPU REQUIRED - Verify CUDA availability
import torch

print("=" * 50)
print("Environment Check")
print("=" * 50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("❌ CUDA not available! This notebook requires a GPU.")

print("=" * 50)
print("✅ GPU environment verified!")


In [ ]:
# Core imports
import os
import json
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple

# Data processing
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from tqdm.auto import tqdm

# NeMo AutoModel imports
try:
    import nemo_automodel
    from nemo_automodel._transformers import NeMoAutoModelForCausalLM
    from nemo_automodel.components._peft.lora import PeftConfig, apply_lora_to_linear_modules
    print(f"✅ NeMo AutoModel imported successfully")
except ImportError as e:
    print(f"⚠️ NeMo AutoModel not found: {e}")
    print("Please install: cd Automodel && uv pip install -e .")
    raise

# Transformers for tokenizer
from transformers import AutoTokenizer, AutoProcessor

# Evaluation
import evaluate

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✅ All imports successful!")


In [ ]:
# Configuration
CONFIG = {
    # Model
    "model_name": "nvidia/Nemotron-Next-8B",
    "torch_dtype": torch.bfloat16,
    
    # Dataset
    "dataset_name": "explodinggradients/fiqa",
    "train_split_ratio": 0.8,  # 80% train, 20% validation from original train
    "max_length": 512,
    
    # LoRA
    "lora_rank": 8,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    
    # Training
    "learning_rate": 2e-4,
    "batch_size": 4,
    "num_epochs": 3,
    "gradient_accumulation_steps": 4,
    "warmup_ratio": 0.1,
    
    # Paths
    "output_dir": "./outputs",
    "checkpoint_dir": "./checkpoints",
}

# Create output directories
Path(CONFIG["output_dir"]).mkdir(parents=True, exist_ok=True)
Path(CONFIG["checkpoint_dir"]).mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")
print(f"   Model: {CONFIG['model_name']}")
print(f"   Dataset: {CONFIG['dataset_name']}")
print(f"   LoRA rank: {CONFIG['lora_rank']}, alpha: {CONFIG['lora_alpha']}")


---

<a name="model-loading"></a>
## 2. Model Loading

**⏱️ Time Estimate: 2-5 minutes** | **GPU REQUIRED**

Load Nemotron Next 8B using NeMo AutoModel APIs.


In [ ]:
# GPU REQUIRED - Load Nemotron Next 8B
# ⏱️ Time: 2-5 minutes

print("=" * 50)
print("Loading Nemotron Next 8B...")
print("=" * 50)

start_time = time.time()

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["model_name"],
    trust_remote_code=True,
)

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✅ Tokenizer loaded")
print(f"   Vocab size: {tokenizer.vocab_size}")
print(f"   Pad token: {tokenizer.pad_token}")

# Load model using NeMo AutoModel
print("\nLoading model (this may take a few minutes)...")
model = NeMoAutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=CONFIG["torch_dtype"],
    trust_remote_code=True,
    device_map="auto",
)

load_time = time.time() - start_time

print(f"\n✅ Model loaded successfully in {load_time:.1f} seconds")
print(f"   Model type: {type(model).__name__}")
print(f"   Dtype: {model.dtype}")
print(f"   Device: {next(model.parameters()).device}")


In [ ]:
# Print model architecture summary
def count_parameters(model):
    """Count trainable and total parameters."""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(model)

print("=" * 50)
print("Model Summary")
print("=" * 50)
print(f"Total parameters: {total:,} ({total/1e9:.2f}B)")
print(f"Trainable parameters: {trainable:,} ({trainable/1e9:.2f}B)")
print(f"Trainable %: {100 * trainable / total:.2f}%")


In [ ]:
# Test basic inference to verify model works
print("=" * 50)
print("Testing Model Inference")
print("=" * 50)

# Simple financial question to test
test_prompt = """You are a helpful financial assistant.

Question: What is the difference between a stock and a bond?

Answer:"""

# Tokenize
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

# Generate
print("Generating response...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n" + "=" * 50)
print("Test Prompt:")
print("=" * 50)
print(test_prompt)
print("\n" + "=" * 50)
print("Model Response:")
print("=" * 50)
print(response[len(test_prompt):].strip())
print("\n✅ Model inference test passed!")


---

<a name="dataset"></a>
## 3. Dataset Loading & Preprocessing

**⏱️ Time Estimate: 10-25 minutes** | **GPU REQUIRED**

Load FiQA dataset, create train/val/test splits, and format for instruction fine-tuning.

> 🚧 **TODO: Phase 3** - Implement dataset loading and preprocessing


---

<a name="baseline"></a>
## 4. Baseline Evaluation

**⏱️ Time Estimate: 30-60 minutes** | **GPU REQUIRED**

Evaluate the base model on FiQA test set before fine-tuning.

> 🚧 **TODO: Phase 4** - Implement baseline evaluation


---

<a name="lora-config"></a>
## 5. LoRA Configuration

**⏱️ Time Estimate: ~5 minutes** | **GPU REQUIRED**

Configure and apply LoRA adapter to the model.

> 🚧 **TODO: Phase 5** - Implement LoRA configuration


---

<a name="training"></a>
## 6. LoRA Training

**⏱️ Time Estimate: 3-6 hours** | **GPU REQUIRED**

Train the LoRA adapter on FiQA training data.

> 🚧 **TODO: Phase 6** - Implement LoRA training


---

<a name="evaluation"></a>
## 7. Fine-tuned Evaluation

**⏱️ Time Estimate: 30-60 minutes** | **GPU REQUIRED**

Evaluate the fine-tuned model and compare with baseline.

> 🚧 **TODO: Phase 7** - Implement fine-tuned evaluation


---

<a name="visualization"></a>
## 8. Visualization & Analysis

Create visualizations comparing baseline vs fine-tuned performance.

> 🚧 **TODO: Phase 8** - Implement visualization and analysis


---

## Summary

This notebook demonstrated LoRA fine-tuning of Nemotron Next 8B on the FiQA financial QA dataset.

### Results

| Metric | Baseline | Fine-tuned | Improvement |
|--------|----------|------------|-------------|
| Exact Match | TBD | TBD | TBD |
| F1 Score | TBD | TBD | TBD |
| BLEU | TBD | TBD | TBD |

### Next Steps

- Experiment with different LoRA ranks
- Try longer training
- Evaluate on additional financial QA datasets
